In [1]:
import laspy
import numpy as np
from scipy.interpolate import griddata
from pyproj import Transformer, CRS
from tqdm import tqdm
import pandas as pd
import sys
import pygmt
from sklearn.cluster import DBSCAN
import utm

In [ ]:
# Define conversion functions

def utm_to_latlon(x, y):
    # Convert lat/lon to UTM coordinates
    lat, lon = utm.to_latlon(x, y, 32, 'U')

    return lat, lon

def latlon_to_utm(lat, lon):
    # Convert lat/lon to UTM coordinates
    utm_x, utm_y, _, _ = utm.from_latlon(lat, lon)

    return utm_x, utm_y

In [369]:
# Example usage
laz_file_path = "../lasersurface/lidar_data/3dm_32_356_5645_1_nw.laz" # Dom

# laz_file_path = "../lasersurface/lidar_data/3dm_32_358_5643_1_nw.laz" # TÜV building in Poll


In [370]:
with laspy.open(laz_file_path) as file:
    las = file.read()

In [372]:
SSFACTOR = 1 # Subsampling factor for points cloud

lastReturnNichtBoden = 20
brueckenpunkte = 17
unclassified = 1

class_ok = [brueckenpunkte, lastReturnNichtBoden, unclassified]

class_val = las.classification[::SSFACTOR]

mask = (np.isin(class_val, class_ok))

points = np.vstack((las.x[::SSFACTOR][mask], las.y[::SSFACTOR][mask], las.z[::SSFACTOR][mask])).transpose()

ground_points = las.points[las.classification == 2]

gnd_points = np.vstack((ground_points.x, ground_points.y, ground_points.z)).transpose()


In [373]:
GCG2016_geoid_file = './GCG2016_data/GCG2016_we.tif'
DEM_file = './DEM_data/urn_eop_DLR_CDEM10_Copernicus_DSM_04_N50_00_E006_00_V8239-2020_1__DEM1__coverage_20231204210410.tif'
egm96_geoid_file = './EGM96_data/egm96_15.gtx'
egg08_geoid_file = './EGG08_data/egm08_25.gtx'

In [374]:
transformer = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids={GCG2016_geoid_file} +multiplier = 1 "  # Vertical grid shift to add the DHHN16 elevation -> WGS84
    f"+step +proj=vgridshift +grids={DEM_file} +multiplier = -1 "  # Vertical grid shift to remove the DEM elevation
    f"+step +proj=vgridshift +grids={egg08_geoid_file} +multiplier = -1 " # Vertical grid shift to remove the EGM2008 geoid height
    #f"+step +proj=vgridshift +grids={egm96_geoid_file} +multiplier = 1 " # Vertical grid shift to add the EGM96 geoid height
)

# Tests

In [109]:
transformed_z = np.array([transformer.transform(xi, yi, zi)[2] for xi, yi, zi in tqdm(zip(las.x, las.y, las.z), total=9471722)])


100%|██████████| 9471722/9471722 [00:37<00:00, 252318.18it/s]


In [108]:
transformer.transform(356525.6,5645288.4,200)

(6.957804522618755, 50.9413476698659, 144.6082798562532)

In [110]:
transformed_z.mean()

2.248118481420427

## test to transform only ground points and check average height

In [111]:
transformed_ground_z = np.array([transformer.transform(xi, yi, zi)[2] for xi, yi, zi in tqdm(zip(ground_points.x, ground_points.y, ground_points.z), total=len(ground_points.x))])

100%|██████████| 3365666/3365666 [00:13<00:00, 253146.69it/s]


In [117]:
transformed_ground_z.mean()

-4.416774161873778

## Check DEM file values with rasterio

In [33]:
import rasterio
from rasterio.transform import from_origin

def get_dem_elevation(dem_file, lat, lon):
    """
    Get the elevation from a DEM file at a given latitude and longitude.

    Parameters:
    dem_file (str): Path to the DEM file.
    lat (float): Latitude of the point.
    lon (float): Longitude of the point.

    Returns:
    float: Elevation at the given latitude and longitude.
    """
    with rasterio.open(dem_file) as dataset:
        # Convert the latitude and longitude to row and column
        row, col = dataset.index(lon, lat)

        # Read the elevation at the given row and column
        elevation = dataset.read(1)[row, col]
        return elevation

In [39]:
latitude, longitude = 50.9392718,6.9659982
elevation = get_dem_elevation(geotiff_file, latitude, longitude)
print(f"Elevation at ({latitude}, {longitude}): {elevation} meters")

Elevation at (50.9392718, 6.9659982): 37.0 meters


# Clustering

In [375]:
#transformed_points = np.array([transformer.transform(xi, yi, zi) for xi, yi, zi in tqdm(zip(las.x, las.y, las.z), total=len(las.x))])

total = len(points[:,0])
transformed_points = np.array([transformer.transform(xi, yi, zi) for xi, yi, zi in tqdm(zip(points[:,0], points[:,1], points[:,2]), total = total)])

100%|██████████| 5614653/5614653 [00:19<00:00, 291185.27it/s]


In [376]:
# Create our dataframe

df = pd.DataFrame(
    data={
        "x": points[:,0], #np.array(las.x), # We need UTM coordinates
        "y": points[:,1], #np.array(las.y), # 
        "z": transformed_points[:,2]
    }
)

size_df = sys.getsizeof(df)
print(f"Size of the DataFrame: {np.ceil(size_df / (1024*1024))} MB")

Size of the DataFrame: 129.0 MB


In [377]:
region = pygmt.info(data=df[["x", "y"]], spacing=1)  # West, East, South, North

print(f"Data points covers region: {region}")

Data points covers region: [ 356000.  357000. 5645000. 5646000.]


In [378]:
df_trimmed = pygmt.blockmedian(
    data=df[["x", "y", "z"]],
    T=0.9999,  # 99.99th quantile, i.e. the highest point
    spacing="1+e", # 1+e for 1 m # 0.1 increases the size of df but more accurate?
    region=region,
)

size_df_trimmed = sys.getsizeof(df_trimmed)
print(f"Size of the DataFrame: {np.ceil(size_df_trimmed / (1024*1024))} MB")

Size of the DataFrame: 16.0 MB


In [381]:
# Identify points that are above 120m above geoid (for Cologne this means about 70 m above ground).
high_points = df_trimmed[df_trimmed['z'] > 70]

# Assuming that points within 100m of each other belong to the same obstacle
clustering = DBSCAN(eps=50, min_samples=2).fit(high_points[['x', 'y', 'z']]) # TODO: no error if no cluster found

# Add the cluster labels to the high_points DataFrame
high_points['cluster'] = clustering.labels_

# Filter out noise points (DBSCAN labels noise as -1)
obstacles = high_points[high_points['cluster'] != -1]

# Find the highest point in each obstacle cluster
highest_points = obstacles.loc[obstacles.groupby('cluster')['z'].idxmax()]

# The resulting DataFrame 'highest_points' contains the coordinates of the highest point of each obstacle
highest_points.reset_index(drop=True, inplace=True)

# Apply the conversion function to the DataFrame to create new columns 'lat' and 'lon'
highest_points['lat'], highest_points['lon'] = zip(*highest_points.apply(lambda row: utm_to_latlon(row['x'], row['y']), axis=1))

# Display the resulting DataFrame
pd.set_option('display.max_rows', 200)
highest_points

/var/folders/v_/0jkxx12x7r9_3szb050fks9m0000gn/T/ipykernel_7563/2073081422.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_points['cluster'] = clustering.labels_


,x,y,z,cluster,lat,lon
0,356490.45,5645300.45,159.007408,0,50.941447,6.957300
1,356119.49,5645270.50,73.419708,1,50.941086,6.952035
2,356149.42,5645192.48,74.105397,2,50.940392,6.952492
3,356215.49,5645201.49,70.924553,3,50.940489,6.953428


## Check the transformation steps to understand their effect

In [367]:
transformer1 = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids={GCG2016_geoid_file} +multiplier = 1 "  # Vertical grid shift to add the DHHN16 elevation -> WGS84
)

transformer2 = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids={DEM_file} +multiplier = -1 "  # Vertical grid shift to remove the DEM elevation
)

transformer3 = Transformer.from_pipeline(
    f"+proj=pipeline "
    f"+step +inv +proj=utm +zone=32 +ellps=WGS84 "  # Convert from UTM Zone 32N to geographic coordinates
    f"+step +proj=vgridshift +grids={egg08_geoid_file} +multiplier = -1 " # Vertical grid shift to remove the EGM2008 geoid height
)

In [380]:
x, y = 356490.45, 5645300.45

print(
    f"Total shift: {transformer.transform(x, y,0)[2]}\n"
    f"Step 1: {transformer1.transform(x, y,0)[2]}\n"
    f"Step 2: {transformer2.transform(x, y,0)[2]}\n"
    f"Step 3: {transformer3.transform(x, y,0)[2]}"
)

Total shift: -53.29106134543536
Step 1: 46.484095331990424
Step 2: -53.06254837438759
Step 3: -46.712608303038195
